<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/5.%20Categorical_Feature_Encoding_Challenge_final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
# !kaggle competitions list

In [ ]:
!kaggle competitions download -c cat-in-the-dat

 42% 9.00M/21.2M [00:00<00:00, 69.4MB/s]
100% 21.2M/21.2M [00:00<00:00, 123MB/s] 


In [ ]:
!ls

cat-in-the-dat.zip  kaggle.json  sample_data


In [ ]:
!unzip cat-in-the-dat.zip

Archive:  cat-in-the-dat.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# **1. 최종 모델 확정**

최종 모델은 로지스틱 회귀 모델입니다. 

# **2. 피처 엔지니어링**

## **2.1. 데이터 준비**

데이터를 준비하겠습니다. 'id' 피처의 고윳값 개수가 레코드 개수와 일치하므로 'id' 피처를 인덱스로 삼겠습니다.

In [ ]:
import numpy as np
import pandas as pd

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 50)
pd.set_option('display.max_rows', 50)

# 데이터를 읽습니다. 'id' 피처는 인덱스로 지정합니다.
train = pd.read_csv('train.csv', index_col='id')
test = pd.read_csv('test.csv', index_col='id')
submission = pd.read_csv('sample_submission.csv', index_col='id')

훈련 세트와 테스트 세트를 위아래로 쌓고 데이터 정보를 확인하겠습니다.

In [ ]:
all_data = pd.concat([train, test], ignore_index=False)
all_data

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0.0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0.0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0.0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1.0
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,0,F,N,Green,Square,Lion,Canada,Theremin,6c90f81cd,a406a5f12,d02a6b0ba,9e4b23160,acc31291f,1,Novice,Lava Hot,j,A,Gb,1,3,NaN
499996,1,0,0,F,Y,Green,Trapezoid,Lion,China,Piano,2d61990e2,520806ce2,d2d8eabdb,cfbd87ed0,eae3446d0,1,Contributor,Lava Hot,f,S,Ed,2,2,NaN
499997,0,1,1,T,Y,Green,Trapezoid,Lion,Canada,Oboe,488406659,28645754b,22831fffe,1108bcd6c,33dd3cf4b,1,Novice,Boiling Hot,g,V,TR,3,1,NaN


In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   bin_0   500000 non-null  int64  
 1   bin_1   500000 non-null  int64  
 2   bin_2   500000 non-null  int64  
 3   bin_3   500000 non-null  object 
 4   bin_4   500000 non-null  object 
 5   nom_0   500000 non-null  object 
 6   nom_1   500000 non-null  object 
 7   nom_2   500000 non-null  object 
 8   nom_3   500000 non-null  object 
 9   nom_4   500000 non-null  object 
 10  nom_5   500000 non-null  object 
 11  nom_6   500000 non-null  object 
 12  nom_7   500000 non-null  object 
 13  nom_8   500000 non-null  object 
 14  nom_9   500000 non-null  object 
 15  ord_0   500000 non-null  int64  
 16  ord_1   500000 non-null  object 
 17  ord_2   500000 non-null  object 
 18  ord_3   500000 non-null  object 
 19  ord_4   500000 non-null  object 
 20  ord_5   500000 non-null  object 
 21  day     50

In [ ]:
def feature_table(df):
    # 데이터 크기를 출력합니다.    
    print(df.shape)
    # 피처 이름과 유형을 담은 데이터프레임을 생성합니다.   
    feature_description = pd.DataFrame(df.dtypes, columns=['type'])
    # 피처 이름 피처가 인덱스이므로 인덱스를 리셋하여 새 인덱스를 만듭니다.
    feature_description = feature_description.reset_index()
    # 피처 이름을 값으로 가진 피처명은 'index'입니다. 'feature'로 변환합니다.
    feature_description = feature_description.rename(columns={'index': 'feature'})
    # 각 피처가 가진 첫 번째 ~ 다섯 번째 값을 출력합니다.
    feature_description['first_value'] = df.loc[0].values
    feature_description['second_value'] = df.loc[1].values
    feature_description['third_value'] = df.loc[2].values
    feature_description['fourth_value'] = df.loc[3].values
    feature_description['fifth_value'] = df.loc[4].values
    # 고윳값 개수를 산출합니다.
    feature_description['eigenvalue_count'] = df.nunique().values
    # 결측치 개수를 산출합니다.
    feature_description['null_count'] = df.isnull().sum().values

    return feature_description

feature_table(train)

(300000, 24)


,feature,type,first_value,second_value,third_value,fourth_value,fifth_value,eigenvalue_count,null_count
0,bin_0,int64,0,0,0,0,0,2,0
1,bin_1,int64,0,1,0,1,0,2,0
2,bin_2,int64,0,0,0,0,0,2,0
3,bin_3,object,T,T,F,F,F,2,0
4,bin_4,object,Y,Y,Y,Y,N,2,0
5,nom_0,object,Green,Green,Blue,Red,Red,3,0
6,nom_1,object,Triangle,Trapezoid,Trapezoid,Trapezoid,Trapezoid,6,0
7,nom_2,object,Snake,Hamster,Lion,Snake,Lion,6,0
8,nom_3,object,Finland,Russia,Russia,Canada,Canada,6,0
9,nom_4,object,Bassoon,Piano,Theremin,Oboe,Oboe,4,0


테스트 세트에는 'target' 피처가 없습니다. 'target' 피처를 제거하겠습니다.

In [ ]:
all_data = all_data.drop('target', axis=1)

## **2.2. bin_3, bin_4**

### **2.2.1. 매핑**

'bin_3', 'bin_4' 피처의 각 값을 숫자로 매핑하겠습니다.

In [ ]:
all_data['bin_3'] = all_data['bin_3'].map({'T': 1, 'F': 0})
all_data['bin_4'] = all_data['bin_4'].map({'Y': 1, 'N': 0})

## **2.3. nom_num**

### **2.3.1. 원-핫 인코딩**

전체 'nom_num' 피처에 `OneHotEncoder` 클래스를 사용하여 원-핫 인코딩을 수행하겠습니다. 물론 (이 데이터에 한정하여) 판다스의 `get_dummies()` 메서드를 사용해도 무방하지만 `get_dummies()` 메서드는 결정적인 단점을 가집니다. 학습 세트의 피처를 학습하는 과정 없이 그대로 원-핫 인코딩을 수행하기 때문에 학습 세트에만 존재하고 테스트 세트에 없는 값은 원-핫 인코딩이 적용된 피처로 변환하지 않습니다. 게다가 `OneHotEncoder` 클래스보다 더 많은 메모리가 필요합니다. `OneHotEncoder` 클래스는 원-핫 인코딩을 수행하여 CSR 형식의 행렬을 반환하기 때문입니다(데이터프레임보다 적은 메모리 소모). 따라서 원-핫 인코딩을 적용하는 경우 습관적으로 `OneHotEncoder` 클래스를 사용하는 편이 좋습니다. [링크](https://psystat.tistory.com/136)에서 상세한 설명을 확인할 수 있습니다.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 전체 'nom_num' 피처를 담은 리스트를 생성합니다.
list_nom_num = ['nom_' + str(i) for i in range(10)]

one_hot_encoder = OneHotEncoder()
# 학습 및 테스트 세트를 하나로 합쳤으므로 `fit_transform()` 메서드를 사용합니다.
# 학습 및 테스트 세트로 나뉜 상태라면 테스트 세트에는 `transform()` 메서드만 사용합니다.
nom_num_one_hot_encoded = one_hot_encoder.fit_transform(all_data[list_nom_num])

In [ ]:
nom_num_one_hot_encoded

<500000x16276 sparse matrix of type '<class 'numpy.float64'>'
	with 5000000 stored elements in Compressed Sparse Row format>

## **2.4. ord_num**

### **2.4.1. 매핑**

순서형 피처는 값 순서에 핵심 의미를 가집니다. 'ord_1', 'ord_2' 피처의 고윳값을 확인해 보겠습니다.

In [ ]:
list_ord_1_2 = ['ord_1', 'ord_2']

for feature in list_ord_1_2:
    print(f'{feature}:\n {all_data[feature].unique()}\n')

ord_1:
 ['Grandmaster' 'Expert' 'Novice' 'Contributor' 'Master']

ord_2:
 ['Cold' 'Hot' 'Lava Hot' 'Boiling Hot' 'Freezing' 'Warm']



값 순서에 알맞도록 숫자로 매핑하겠습니다.

In [ ]:
value_sequence_ord_1 = {'Novice':0, 'Contributor': 1, 'Expert': 2,
                  'Master': 3, 'Grandmaster': 4}
value_sequence_ord_2 = {'Freezing': 0, 'Cold': 1, 'Warm': 2, 'Hot': 3,
                  'Boiling Hot': 4, 'Lava Hot': 5}

all_data['ord_1'] = all_data['ord_1'].map(value_sequence_ord_1)
all_data['ord_2'] = all_data['ord_2'].map(value_sequence_ord_2)

'ord_3 ~ 5' 피처의 고윳값을 확인해 보겠습니다.

In [ ]:
list_ord_3_4_5 = ['ord_3', 'ord_4', 'ord_5']

for feature in list_ord_3_4_5:
    print(f'{feature}:\n {all_data[feature].unique()}\n')

ord_3:
 ['h' 'a' 'i' 'j' 'g' 'e' 'd' 'b' 'k' 'f' 'l' 'n' 'o' 'c' 'm']

ord_4:
 ['D' 'A' 'R' 'E' 'P' 'K' 'V' 'Q' 'Z' 'L' 'F' 'T' 'U' 'S' 'Y' 'B' 'H' 'J'
 'N' 'G' 'W' 'I' 'O' 'C' 'X' 'M']

ord_5:
 ['kr' 'bF' 'Jc' 'kW' 'qP' 'PZ' 'wy' 'Ed' 'qo' 'CZ' 'qX' 'su' 'dP' 'aP'
 'MV' 'oC' 'RL' 'fh' 'gJ' 'Hj' 'TR' 'CL' 'Sc' 'eQ' 'kC' 'qK' 'dh' 'gM'
 'Jf' 'fO' 'Eg' 'KZ' 'Vx' 'Fo' 'sV' 'eb' 'YC' 'RG' 'Ye' 'qA' 'lL' 'Qh'
 'Bd' 'be' 'hT' 'lF' 'nX' 'kK' 'av' 'uS' 'Jt' 'PA' 'Er' 'Qb' 'od' 'ut'
 'Dx' 'Xi' 'on' 'Dc' 'sD' 'rZ' 'Uu' 'sn' 'yc' 'Gb' 'Kq' 'dQ' 'hp' 'kL'
 'je' 'CU' 'Fd' 'PQ' 'Bn' 'ex' 'hh' 'ac' 'rp' 'dE' 'oG' 'oK' 'cp' 'mm'
 'vK' 'ek' 'dO' 'XI' 'CM' 'Vf' 'aO' 'qv' 'jp' 'Zq' 'Qo' 'DN' 'TZ' 'ke'
 'cG' 'tP' 'ud' 'tv' 'aM' 'xy' 'lx' 'To' 'uy' 'ZS' 'vy' 'ZR' 'AP' 'GJ'
 'Wv' 'ri' 'qw' 'Xh' 'FI' 'nh' 'KR' 'dB' 'BE' 'Bb' 'mc' 'MC' 'tM' 'NV'
 'ih' 'IK' 'Ob' 'RP' 'dN' 'us' 'dZ' 'yN' 'Nf' 'QM' 'jV' 'sY' 'wu' 'SB'
 'UO' 'Mx' 'JX' 'Ry' 'Uk' 'uJ' 'LE' 'ps' 'kE' 'MO' 'kw' 'yY' 'zU' 'bJ'
 'Kf' 'ck' 'mb' 'Os' 'Ps

고윳값 개수가 많기 때문에 일일이 매핑하기는 번거롭습니다. `OrdinalEncoder` 클래스를 활용하여 값 순서에 맞게 숫자로 매핑하겠습니다.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ord_encoder = OrdinalEncoder()
all_data[list_ord_3_4_5] = ord_encoder.fit_transform(all_data[list_ord_3_4_5])

아래와 같이 값 순서가 `categories_` 속성에 저장되고, 각 값은 값 순서에 맞게 숫자로 매핑됩니다.

In [ ]:
ord_encoder.categories_

[array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
        'n', 'o'], dtype=object),
 array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
        'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
       dtype=object),
 array(['AP', 'Ai', 'Aj', 'BA', 'BE', 'Bb', 'Bd', 'Bn', 'CL', 'CM', 'CU',
        'CZ', 'Cl', 'DH', 'DN', 'Dc', 'Dx', 'Ed', 'Eg', 'Er', 'FI', 'Fd',
        'Fo', 'GD', 'GJ', 'Gb', 'Gx', 'Hj', 'IK', 'Id', 'JX', 'Jc', 'Jf',
        'Jt', 'KR', 'KZ', 'Kf', 'Kq', 'LE', 'MC', 'MO', 'MV', 'Mf', 'Ml',
        'Mx', 'NV', 'Nf', 'Nk', 'OR', 'Ob', 'Os', 'PA', 'PQ', 'PZ', 'Ps',
        'QM', 'Qb', 'Qh', 'Qo', 'RG', 'RL', 'RP', 'Rm', 'Ry', 'SB', 'Sc',
        'TR', 'TZ', 'To', 'UO', 'Uk', 'Uu', 'Vf', 'Vx', 'WE', 'Wc', 'Wv',
        'XI', 'Xh', 'Xi', 'YC', 'Yb', 'Ye', 'ZR', 'ZS', 'Zc', 'Zq', 'aF',
        'aM', 'aO', 'aP', 'ac', 'av', 'bF', 'bJ', 'be', 'cA', 'cG', 'cW',
        'ck', 'cp', 'dB', 'dE', 'dN', 'dO', 'dP', 'dQ', 'd

In [ ]:
for feature in list_ord_3_4_5:
    print(f'{feature}:\n {all_data[feature].unique()}\n')

ord_3:
 [ 7.  0.  8.  9.  6.  4.  3.  1. 10.  5. 11. 13. 14.  2. 12.]

ord_4:
 [ 3.  0. 17.  4. 15. 10. 21. 16. 25. 11.  5. 19. 20. 18. 24.  1.  7.  9.
 13.  6. 22.  8. 14.  2. 23. 12.]

ord_5:
 [136.  93.  31. 134. 158.  53. 185.  17. 160.  11. 159. 170. 105.  90.
  41. 147.  60. 116. 117.  27.  66.   8.  65. 110. 129. 157. 108. 118.
  32. 115.  18.  35.  73.  22. 167. 111.  80.  59.  82. 155. 139.  57.
   6.  95. 120. 138. 145. 131.  92. 175.  33.  51.  19.  56. 151. 178.
  16.  79. 152.  15. 166. 163.  71. 169. 190.  25.  37. 106. 122. 132.
 127.  10.  21.  52.   7. 114. 121.  91. 165. 102. 148. 150. 100. 144.
 181. 113. 104.  77.   9.  72.  89. 161. 128.  86.  58.  14.  67. 135.
  97. 172. 176. 173.  88. 187. 141.  68. 180.  84. 183.  83.   0.  24.
  76. 164. 162.  78.  20. 146.  34. 101.   4.   5. 143.  39. 171.  45.
 124.  28.  49.  61. 103. 177. 107. 188.  46.  55. 126. 168. 184.  64.
  69.  44.  30.  63.  70. 174.  38. 154. 130.  40. 137. 189. 191.  94.
  36.  99. 142.  50.  54

### **2.4.2. min-max 정규화**

min-max 정규화를 수행하겠습니다.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

list_ord_num = ['ord_' + str(i) for i in range(6)]

min_max_scaler = MinMaxScaler()
all_data[list_ord_num] = min_max_scaler.fit_transform(all_data[list_ord_num])

In [ ]:
for feature in list_ord_num:
    print(f'{feature}:\n {all_data[feature].unique()}\n')

ord_0:
 [0.5 0.  1. ]

ord_1:
 [1.   0.5  0.   0.25 0.75]

ord_2:
 [0.2 0.6 1.  0.8 0.  0.4]

ord_3:
 [0.5        0.         0.57142857 0.64285714 0.42857143 0.28571429
 0.21428571 0.07142857 0.71428571 0.35714286 0.78571429 0.92857143
 1.         0.14285714 0.85714286]

ord_4:
 [0.12 0.   0.68 0.16 0.6  0.4  0.84 0.64 1.   0.44 0.2  0.76 0.8  0.72
 0.96 0.04 0.28 0.36 0.52 0.24 0.88 0.32 0.56 0.08 0.92 0.48]

ord_5:
 [0.71204188 0.48691099 0.16230366 0.70157068 0.82722513 0.27748691
 0.96858639 0.08900524 0.83769634 0.05759162 0.83246073 0.89005236
 0.54973822 0.47120419 0.21465969 0.76963351 0.31413613 0.60732984
 0.61256545 0.14136126 0.34554974 0.04188482 0.34031414 0.57591623
 0.67539267 0.82198953 0.56544503 0.61780105 0.16753927 0.60209424
 0.09424084 0.18324607 0.38219895 0.11518325 0.87434555 0.58115183
 0.41884817 0.30890052 0.42931937 0.81151832 0.72774869 0.29842932
 0.03141361 0.4973822  0.62827225 0.72251309 0.7591623  0.68586387
 0.48167539 0.91623037 0.17277487 0.267015

## **2.5. day, month**

'day', 'month' 피처에는 원-핫 인코딩을 적용하겠습니다. 순서형 피처로 간주하여 min-max 정규화를 적용해 볼 수도 있습니다.

In [ ]:
list_date = ['day', 'month']

date_one_hot_encoded = one_hot_encoder.fit_transform(all_data[list_date])

In [ ]:
date_one_hot_encoded

<500000x19 sparse matrix of type '<class 'numpy.float64'>'
	with 1000000 stored elements in Compressed Sparse Row format>

## **2.6. 불필요한 피처 제거**

대부분의 피처 엔지니어링을 마쳤습니다. 원-핫 인코딩을 적용한 피처들은 CSR 형식으로 따로 저장된 상태이므로 기존 'nom_num', 'day', 'month' 피처는 불필요합니다. 삭제하겠습니다.

In [ ]:
list_nom_num = ['nom_' + str(i) for i in range(10)]
list_date = ['day', 'month']

for features in [list_nom_num, list_date]:
    all_data = all_data.drop(features, axis=1)

## **2.7. 데이터 세트와 인코딩된 피처 간 결합**

데이터 세트와 인코딩된 피처를 결합하겠습니다. 기존 데이터를 CSR 형식으로 변환하여 인코딩된 CSR 행렬과 결합하고, 메모리를 절약하기 위해 CSR 형식을 그대로 유지시키겠습니다. 

In [ ]:
from scipy import sparse

all_data_sparse = sparse.hstack([sparse.csr_matrix(all_data), # 기존 데이터를 CSR 형식으로 변환합니다.
                                 nom_num_one_hot_encoded,
                                 date_one_hot_encoded], format='csr')

In [ ]:
all_data_sparse

<500000x16306 sparse matrix of type '<class 'numpy.float64'>'
	with 9163718 stored elements in Compressed Sparse Row format>

# **4. 최종 모델링 및 하이퍼파라미터 튜닝**

## **4.1. 데이터 준비**

데이터를 준비하겠습니다.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_sparse, y_train_original, test_size=0.1, stratify=y_train_original, random_state=2023)

## **4.2. LogisticRegression**

### **4.2.1. optuna**

In [ ]:
def objective_LogisticRegression(trial, X_train, y_train, X_valid, y_valid):
    params = {
        'C': trial.suggest_float('C', 0.1, 10),
        'max_iter': trial.suggest_int('max_iter', 100, 10000),
        'solver': 'liblinear',
        'random_state': 2023
    }

    model = LogisticRegression(**params)
    model.fit(X_train, y_train)
    prediction = model.predict_proba(X_valid)[:, 1]
    roc_auc = roc_auc_score(y_valid, prediction)

    return roc_auc

In [ ]:
study_logistic = optuna.create_study(direction='maximize', sampler=TPESampler())
study_logistic.optimize(lambda trial: objective_LogisticRegression(trial, X_train, y_train, X_valid, y_valid),
                        n_trials=500, show_progress_bar=True)

print(f'Best trial score: {study_logistic.best_trial.value}')
print(f'Best params: {study_logistic.best_trial.params}')

[I 2023-01-22 18:36:21,923] A new study created in memory with name: no-name-fbb0da3e-54d0-4deb-9202-de293c1ffd40
/usr/local/lib/python3.8/dist-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2023-01-22 18:36:56,247] Trial 0 finished with value: 0.7891953153862101 and parameters: {'C': 7.578692506041821, 'max_iter': 9286}. Best is trial 0 with value: 0.7891953153862101.
[I 2023-01-22 18:37:19,258] Trial 1 finished with value: 0.7942987002204887 and parameters: {'C': 1.662506209505343, 'max_iter': 1276}. Best is trial 1 with value: 0.7942987002204887.
[I 2023-01-22 18:37:50,067] Trial 2 finished with value: 0.7885333867584884 and parameters: {'C': 9.84151812533324, 'max_iter': 2442}. Best is trial 1 with value: 0.7942987002204887.
[I 2023-01-22 18:38:19,891] Trial 3 finished with value: 0.7892453726854636 and parameters: {'C': 7.410787242838828, 'max_iter': 7746}. Best is trial 1 with value: 0.7942987002204887.
[I 2023-01-22 18:38:45,182] Trial 4 finished with value: 0.78979574654153 and parameters: {'C': 6.009764251362852, 'max_iter': 1171}. Best is trial 1 with value: 0.7942987002204887.
[I 2023-01-22 18:39:11,241] Trial 5 finished with value: 0.7902620670485836 and par

탐색 211회에 최적 하이퍼파라미터를 찾았습니다. 전체 데이터를 사용하여 모델을 훈련해 보겠습니다.

In [ ]:
params = study_logistic.best_trial.params
model_logistic = LogisticRegression(**params)
model_logistic.fit(X_train_sparse, y_train_original)

LogisticRegression(C=0.10834540601334458, max_iter=5585)

파일을 제출해 보겠습니다.

In [ ]:
submission['target'] = model_logistic.predict_proba(X_test_sparse)[:, 1]
submission.to_csv('Categorical_Feature_Encoding_Challenge_submission11.csv')

프라이빗 스코어는 0.80276입니다. 1338팀 중 14위에 위치한 기록입니다.

# **5. 결론**

XGBoost와 LightGBM의 최적 하이퍼파라미터 탐색에 성공했다면 로지스틱 회귀보다 더 나은 성능을 보였을 것입니다. 실전 대회라면 좋은 컴퓨팅 자원을 확보하고 체계적으로 스케줄을 관리하여 최적 하이퍼파라미터를 찾는 데 전념해 볼 필요가 있습니다.

물론 최적 하이퍼파라미터를 찾는 작업보다 더 중요한 것은 피처 엔지니어링입니다. 주어진 시간이 충분하지 않은 경우 택할 수 있는 전략적인 방법입니다. 최적 하이퍼파라미터를 찾기 위해 많은 시간을 투입하기 보다는 피처 엔지니어링에 더욱 중점을 두는 것입니다. 후자의 작업이 성능을 향상시키는 데 효과적입니다. 

여기에 두 모델을 앙상블하여 약간의 점수라도 향상시킬 수 있습니다. 물론 항상 점수가 향상되는 것은 아닙니다. 